In [1]:


import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context
    
    


# 1. Download and Explore Dataset

In [2]:
#download all the dependencies that we will need.

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy

!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

#print('Libraries imported.')


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
#keep this cell, content should be correct.

import pandas as pd 

import requests 

from bs4 import BeautifulSoup 

headers = { 'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36", } 
response = requests.request("GET", "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", headers=headers, verify=False)

#req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 

soup = BeautifulSoup(response.content,'lxml') 

#table = soup.find_all('table')[0]  

#df = pd.read_html(str(table)) 

#neighborhood=pd.DataFrame(df[0]) 


table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)

neighborhoods=pd.DataFrame(table_contents)
neighborhoods['Borough']=neighborhoods['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


In [6]:
!pip install geocoder
import geocoder

latitude=[]
longitude=[]
for code in neighborhoods['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    #print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        #print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

In [7]:
# add latitude and longitude to Neighbourhoods Table
neighborhoods['Latitude']=latitude
neighborhoods['Longitude']=longitude
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [102]:
# create map of Toronto using latitude and longitude values
import folium

latitude = 43.6532
longitude = -79.3832

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [111]:
NYork_data = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
NYork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
4,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493


In [112]:
# create map of Downtown Toronto using latitude and longitude values
map_NYork = folium.Map(location=[43.65512, -79.36264], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NYork_data['Latitude'], NYork_data['Longitude'], NYork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NYork)  
    
map_NYork

In [113]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2' # your Foursquare ID
CLIENT_SECRET = 'TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2
CLIENT_SECRET:TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV


In [114]:
#Get the neighborhood's name.
NYork_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [115]:
#Get the neighborhood's latitude and longitude values.

neighborhood_latitude = NYork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = NYork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = NYork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Regent Park, Harbourfront are 43.65512000000007, -79.36263999999994.


In [116]:
#get the top 100 venues that are in Parkwoods within a radius of 500 meters.


LIMIT = 100 # limit of number of venues returned by Foursquare API


radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    'B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2', 
    'TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV', 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2&client_secret=TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV&v=20180605&ll=43.65512000000007,-79.36263999999994&radius=500&limit=100'

In [117]:
#Send the GET request and examine the resutls

import requests 

#results1=requests.request("GET", 'https://api.foursquare.com/v2/venues/explore?&client_id=B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2&client_secret=TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV&v=20180605&ll=43.75245000000007,-79.32990999999998&radius=500&limit=100', headers=headers, verify=False)
#results1.json()

url='https://api.foursquare.com/v2/venues/explore?&client_id=B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2&client_secret=TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV&v=20180605&ll=43.75245000000007,-79.32990999999998&radius=500&limit=100'
results=requests.request("GET", url, headers=headers, verify=False).json()
#results.json()


C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [118]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [119]:
#Now we are ready to clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-119-b933ff550a39>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [120]:
#And how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


# 2. Explore Neighborhoods in Downtown Toronto

In [121]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2', 
            'TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV', 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()['response']['groups'][0]['items']
        results=requests.request("GET", url, headers=headers, verify=False).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [122]:
#Now write the code to run the above function on each neighborhood and create a new dataframe called NYork_venues.

NYork_venues = getNearbyVenues(names=NYork_data['Neighborhood'],
                                   latitudes=NYork_data['Latitude'],
                                   longitudes=NYork_data['Longitude'],
                                   radius=500
                                  )

Regent Park, Harbourfront


C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Garden District, Ryerson


C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


St. James Town


...

C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


First Canadian Place, Underground city


C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Church and Wellesley


C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [123]:
print(NYork_venues.shape)
NYork_venues.head()

(1049, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65512,-79.36264,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
3,"Regent Park, Harbourfront",43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space
4,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot


In [124]:
NYork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,48,48,48,48,48,48
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",75,75,75,75,75,75
Central Bay Street,55,55,55,55,55,55
Christie,9,9,9,9,9,9
Church and Wellesley,67,67,67,67,67,67
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",53,53,53,53,53,53


In [125]:
print('There are {} uniques categories.'.format(len(NYork_venues['Venue Category'].unique())))

There are 175 uniques categories.


# 3. Analyze Each Neighborhood

In [126]:
# one hot encoding
NYork_onehot = pd.get_dummies(NYork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NYork_onehot['Neighborhood'] = NYork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NYork_onehot.columns[-1]] + list(NYork_onehot.columns[:-1])
NYorkn_onehot = NYork_onehot[fixed_columns]

NYork_onehot.head()

,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [127]:
NYork_onehot.shape

(1049, 175)

In [128]:
NYork_grouped = NYork_onehot.groupby('Neighborhood').mean().reset_index()
NYork_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.020833,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.062500,0.020833,0.000000,0.000000,0.041667,0.000000,0.000000,0.00,0.020833,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.020833,0.000000,0.000000,0.020833,0.000000,0.000000,0.083333,0.041667,0.000000,0.000000,0.00,0.00,0.020833,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.00,0.020833,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.020833,0.020833,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.020833,0.000000,0.000000,0.020833,0.000000,0.020833,0.000000,0.000000,0.020833,0.000000,0.000000,0.00,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.062500,0.041667,0.000000,0.000000,0.000000,0.00,0.020833,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,

In [129]:
NYork_grouped.shape

(17, 175)

In [130]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in NYork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NYork_grouped[NYork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0        Cocktail Bar  0.08
1      Sandwich Place  0.06
2              Bakery  0.06
3  Seafood Restaurant  0.04
4         Coffee Shop  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.07
2      Sandwich Place  0.05
3                Park  0.05
4   French Restaurant  0.04


----Central Bay Street----
              venue  freq
0       Coffee Shop  0.11
1    Clothing Store  0.07
2       Pizza Place  0.05
3    Sandwich Place  0.05
4  Sushi Restaurant  0.05


----Christie----
                venue  freq
0                Café  0.33
1       Grocery Store  0.22
2         Coffee Shop  0.11
3  Italian Restaurant  0.11
4  Athletics & Sports  0.11


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1     Sushi Restaurant  0.07
2  Japanese Restaurant  0.07
3       

In [131]:
#Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [132]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NYork_grouped['Neighborhood']

for ind in np.arange(NYork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NYork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Sandwich Place,Bakery,Seafood Restaurant,Coffee Shop,Farmers Market,Vegetarian / Vegan Restaurant,Beer Bar,Butcher,Café
1,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Sandwich Place,Park,French Restaurant,Restaurant,Bar,Gym / Fitness Center,Speakeasy,Bakery
2,Central Bay Street,Coffee Shop,Clothing Store,Pizza Place,Sandwich Place,Sushi Restaurant,Plaza,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Bank
3,Christie,Café,Grocery Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Baby Store,New American Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Gym,Burrito Place,Fast Food Restaurant,Grocery Store,Dance Studio


# 4. Cluster Neighborhoods

In [133]:
#import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

NYork_grouped_clustering = NYork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NYork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 3, 0, 0, 0, 0, 0, 2])

In [134]:
kmeans.labels_

array([2, 2, 0, 3, 0, 0, 0, 0, 0, 2, 4, 0, 1, 0, 2, 0, 2])

In [136]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
#neighborhoods_venues_sorted.pop('Cluster Labels')
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NYork_merged = NYork_data

# merge NYork_grouped with NYork_data to add latitude/longitude for each neighborhood
NYork_merged = NYork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')

NYork_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,2,Cocktail Bar,Sandwich Place,Bakery,Seafood Restaurant,Coffee Shop,Farmers Market,Vegetarian / Vegan Restaurant,Beer Bar,Butcher,Café
12,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.64082,-79.39818,2,Italian Restaurant,Coffee Shop,Sandwich Place,Park,French Restaurant,Restaurant,Bar,Gym / Fitness Center,Speakeasy,Bakery
4,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493,0,Coffee Shop,Clothing Store,Pizza Place,Sandwich Place,Sushi Restaurant,Plaza,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Bank
5,M6G,Downtown Toronto,Christie,43.66869,-79.42071,3,Café,Grocery Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Baby Store,New American Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
16,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Gym,Burrito Place,Fast Food Restaurant,Grocery Store,Dance Studio


In [137]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NYork_merged['Latitude'], NYork_merged['Longitude'], NYork_merged['Neighborhood'], NYork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine Clusters

In [138]:
#Cluster 1

NYork_merged.loc[NYork_merged['Cluster Labels'] == 0, NYork_merged.columns[[1] + list(range(5, NYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Pizza Place,Sandwich Place,Sushi Restaurant,Plaza,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Bank
16,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Gym,Burrito Place,Fast Food Restaurant,Grocery Store,Dance Studio
9,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Sandwich Place,Beer Bar,Cocktail Bar,Gym,Japanese Restaurant,Italian Restaurant
15,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Hotel,Japanese Restaurant,Bank,Restaurant,Asian Restaurant,Deli / Bodega,Gym
1,Downtown Toronto,0,Coffee Shop,Sandwich Place,Clothing Store,Hotel,Café,Thai Restaurant,Pizza Place,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant
7,Downtown Toronto,0,Coffee Shop,Sandwich Place,Japanese Restaurant,Boat or Ferry,Bank,Hotel,Plaza,Park,Aquarium,Cocktail Bar
6,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Hotel,Sushi Restaurant,Japanese Restaurant,Restaurant,Asian Restaurant,Bank,Deli / Bodega
2,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Italian Restaurant,Café,Clothing Store,Cosmetics Shop,Restaurant,Gastropub,Gym,Diner
8,Downtown Toronto,0,Coffee Shop,Café,Hotel,Sandwich Place,Asian Restaurant,Deli / Bodega,Restaurant,Steakhouse,Salad Place,Japanese Restaurant


In [139]:
NYork_merged.loc[NYork_merged['Cluster Labels'] == 1, NYork_merged.columns[[1] + list(range(5, NYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,1,Park,Playground,Bike Trail,Adult Boutique,Museum,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant


In [140]:
NYork_merged.loc[NYork_merged['Cluster Labels'] == 2, NYork_merged.columns[[1] + list(range(5, NYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,2,Cocktail Bar,Sandwich Place,Bakery,Seafood Restaurant,Coffee Shop,Farmers Market,Vegetarian / Vegan Restaurant,Beer Bar,Butcher,Café
12,Downtown Toronto,2,Italian Restaurant,Coffee Shop,Sandwich Place,Park,French Restaurant,Restaurant,Bar,Gym / Fitness Center,Speakeasy,Bakery
11,Downtown Toronto,2,Café,Pizza Place,Art Gallery,Vegetarian / Vegan Restaurant,Thai Restaurant,Burger Joint,Gaming Cafe,Comfort Food Restaurant,Park,Grocery Store
14,Downtown Toronto,2,Coffee Shop,Bakery,Italian Restaurant,Restaurant,Park,Café,Pizza Place,Pub,Butcher,Sandwich Place
10,Downtown Toronto,2,Café,Coffee Shop,Bakery,Restaurant,Gym,Bar,Pub,Sandwich Place,Japanese Restaurant,Cheese Shop


In [141]:
NYork_merged.loc[NYork_merged['Cluster Labels'] == 3, NYork_merged.columns[[1] + list(range(5, NYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,3,Café,Grocery Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Baby Store,New American Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant


In [142]:
NYork_merged.loc[NYork_merged['Cluster Labels'] == 4, NYork_merged.columns[[1] + list(range(5, NYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Restaurant,Breakfast Spot,Greek Restaurant,Sandwich Place,Spa,Sushi Restaurant,Pub,Thai Restaurant,Bakery
